In [164]:
Competition submission (for the second day on this case)
Now you will need to use your best model to predict on the test set. To do this, you will need to take the following steps:

Decide on which model, which parameters for the model, and what data to train the model based on your analysis above.
Go back and fit your model on the entire elect_df_train dataframe (not just the elect_df_smaller_train dataframe).
Transform your elect_df_test dataframe in the same way that you did the elect_df_train dataframe.
Predict on your transformed elect_df_test dataframe using the predict() method of your chosen model.
Save the predictions in a variable called bm_all_fit_pred (it should have 1131 rows and 1 column).
Run the code below to generate your submission csv (change TeamX to your team).
A useful guiding principle when coming up with your final prediction is that you should copy your code above (maybe even to a new notebook) and everywhere you see smaller_train replace it with train and everywhere you see validation replace it with test. This will include potentially redoing imputation and feature engineering steps as well. You should not be doing any model selection during this process, but just refitting the model you selected with the full training data set and then predicting on the testing set (just like you predicted on the validation set). However, you will not be able to test your model's accuracy using the accuracy function because you do not have the realizations for the test set.

The below code will combine the row numbers from the original elect_df_test dataset and the predictions from bm_all_fit_pred into a new dataframe called submission_df. NOTE: Until you have a variable called bm_all_fit_pred with your predictions in it, you will receive an error.


submission_df = pd.DataFrame(bm_all_fit_pred, index=elect_df_test.index, columns=['Obama_margin_percent'])
You now should have a dataframe that has indices equal to the indices for elect_df_test going from 1737 to 2867 and a column titled Obama_margin_percent. Check to make sure this is the case:


submission_df
This can then be written to a submission file that is in the correct format for uploading to Kaggle.


submission_df.to_csv("ObamaSubmissionTeamX.csv", index_label='Row')
Sample Code for Getting Final Predictions
Below is sample code (not intended to be used for your actual submission) walking you through the above steps for the case where we choose the untuned decision tree. Note that the major difference is that we are using the entire training data set to create our design matrices (and not just elect_df_smaller_train). Then we are predicting on the test set. We cannot check our accuracy on the test set because we don't have the realizations yet.

However, before we can even train our model, we need to make sure that we re-impute using an imputer trained on the full training set (remember before we just trained the imputer on elect_df_smaller_train). So, we create a new imputer:


final_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

final_imputer.fit(elect_df_train.loc[:, "MalesPer100Females":"FarmArea"])
Once we have fit the imputer (note that we fit all of the numeric rows at once, we don't need an imputer per column), we can then transform the data.


elect_df_train.loc[:, "MalesPer100Females":"FarmArea"] = final_imputer.transform(elect_df_train.loc[:, "MalesPer100Females":"FarmArea"])
We can also impute on our testing data set as well using our final_imputer.


elect_df_test.loc[:, "MalesPer100Females":"FarmArea"] = final_imputer.transform(elect_df_test.loc[:, "MalesPer100Females":"FarmArea"])
Now we can build our final model following the same steps that we did above (note that if you are choosing a different model or you used different variables, this process will be slightly different).


variables_final = [variable for variable in elect_df_smaller_train.columns if variable not in ['FIPS', 'County', 'State', 'ElectionDate', 'TotalVote', 'Obama_margin', 'Obama_margin_percent', 'Obama_wins', 'Clinton', 'Obama']]

formula_final = 'Obama_margin_percent ~ ' + " + ".join(variables[:2]) +  " + standardize(" + ") + standardize(".join(variables[2:]) + ")"

y_linear_final, X_linear_final = dmatrices(formula_final, elect_df_train, return_type="dataframe")

lm_lasso = Lasso(alpha = 1, max_iter=1000000)

lm_lasso.fit(X_linear_final, y_linear_final)

X_linear_test = build_design_matrices([X_linear_final.design_info], elect_df_test, return_type="dataframe")[0]
Now that we have our model built and our data transformed, we can make our final predictions and save them in bm_all_fit_pred.


bm_all_fit_pred = lm_lasso.predict(X_linear_test)

bm_all_fit_pred.shape
We can see that our data is in the right shape.

SyntaxError: unterminated string literal (detected at line 10) (639713757.py, line 10)

In [214]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib as mpl
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split, GridSearchCV
import statsmodels.formula.api as smf

from patsy import dmatrices, dmatrix, build_design_matrices

import math

In [215]:
# This sets some nicer defaults for plotting.
# This must be run in a separate cell from importing matplotlib due to a bug.
params = {'legend.fontsize': 'large',
          'figure.figsize': (11.0, 11.0),
          'axes.labelsize': 'x-large',
          'axes.titlesize':'xx-large',
          'xtick.labelsize':'large',
          'ytick.labelsize':'large'}
mpl.rcParams.update(params)

# This makes it so that the pandas dataframes don't get truncated horizontally.
pd.options.display.max_columns = 200

In [216]:
elect_df = pd.read_csv("Obama.csv")

In [217]:
elect_df.columns

Index(['County', 'State', 'Region', 'FIPS', 'ElectionDate', 'ElectionType',
       'TotalVote', 'Clinton', 'Obama', 'MalesPer100Females', 'AgeBelow35',
       'Age35to65', 'Age65andAbove', 'White', 'Black', 'Asian',
       'AmericanIndian', 'Hawaiian', 'Hispanic', 'HighSchool', 'Bachelors',
       'Poverty', 'IncomeAbove75K', 'MedianIncome', 'AverageIncome',
       'UnemployRate', 'ManfEmploy', 'SpeakingNonEnglish', 'Medicare',
       'MedicareRate', 'SocialSecurity', 'SocialSecurityRate',
       'RetiredWorkers', 'Disabilities', 'DisabilitiesRate', 'Homeowner',
       'SameHouse1995and2000', 'Pop', 'PopDensity', 'LandArea', 'FarmArea'],
      dtype='object')

In [218]:
elect_df.shape

(2868, 41)

In [219]:
elect_df.head()

,County,State,Region,FIPS,ElectionDate,ElectionType,TotalVote,Clinton,Obama,MalesPer100Females,AgeBelow35,Age35to65,Age65andAbove,White,Black,Asian,AmericanIndian,Hawaiian,Hispanic,HighSchool,Bachelors,Poverty,IncomeAbove75K,MedianIncome,AverageIncome,UnemployRate,ManfEmploy,SpeakingNonEnglish,Medicare,MedicareRate,SocialSecurity,SocialSecurityRate,RetiredWorkers,Disabilities,DisabilitiesRate,Homeowner,SameHouse1995and2000,Pop,PopDensity,LandArea,FarmArea
0,Adair,IA,Midwest,19001,1/3/2008,Caucuses,75.0,22.0,24.0,96.7,37.7,40.3,21.9,99.2,0.1,0.5,NaN,0.0,0.8,87.8,11.2,9.7,8.6,39568.0,29222.0,3.2,12.916739,2.0,1551.0,19735.0,1825.0,23222.0,1230.0,90.0,1145.0,75.0,66.0,7714,13.5,570.0,373.0
1,Adams,IA,Midwest,19003,1/3/2008,Caucuses,50.0,18.0,7.0,96.8,37.1,40.8,22.1,99.2,NaN,0.2,0.5,0.0,0.6,84.5,12.0,10.6,7.8,35296.0,27297.0,4.6,6.964421,1.6,978.0,22936.0,1110.0,26032.0,720.0,78.0,1829.0,74.7,65.4,4192,9.9,425.0,238.0
2,Allamakee,IA,Midwest,19005,1/3/2008,Caucuses,80.0,25.0,33.0,104.5,41.5,40.8,17.7,99.0,0.2,0.2,0.3,0.0,6.9,81.4,14.4,10.8,10.7,36822.0,25579.0,4.6,16.473466,8.2,2838.0,19294.0,3270.0,22231.0,2245.0,186.0,1265.0,76.4,64.1,14796,23.1,659.0,326.0
3,Appanoose,IA,Midwest,19007,1/3/2008,Caucuses,60.0,17.0,10.0,94.0,42.0,39.2,18.9,98.5,0.7,0.4,0.1,0.0,1.1,81.4,12.2,14.9,6.6,31764.0,23705.0,5.4,15.080881,2.5,3072.0,22479.0,3415.0,24989.0,2050.0,453.0,3315.0,74.0,60.2,13422,27.0,516.0,236.0
4,Audubon,IA,Midwest,19009,1/3/2008,Caucuses,48.0,16.0,17.0,94.7,37.4,40.2,22.5,99.3,0.2,0.2,0.1,0.0,0.6,82.5,12.3,9.2,9.7,38941.0,31100.0,4.1,8.333333,2.7,1472.0,22797.0,1740.0,26947.0,1150.0,79.0,1223.0,79.2,70.4,6278,14.2,444.0,261.0


In [220]:
def summarize_dataframe(df):
    missing_values = pd.concat([pd.DataFrame(df.columns, columns=['Variable Name']), 
                      pd.DataFrame(df.dtypes.values.reshape([-1,1]), columns=['Data Type']),
                      pd.DataFrame(df.isnull().sum().values, columns=['Missing Values']), 
                      pd.DataFrame([df[name].nunique() for name in df.columns], columns=['Unique Values'])], 
                     axis=1).set_index('Variable Name')
    return pd.concat([missing_values, df.describe(include='all').transpose()], axis=1).fillna("")

In [221]:
summarize_dataframe(elect_df)

,Data Type,Missing Values,Unique Values,count,unique,top,freq,mean,std,min,25%,50%,75%,max
County,object,0,1691,2868.0,1691,Washington,30,,,,,,,
State,object,0,46,2868.0,46,TX,251,,,,,,,
Region,object,0,4,2868.0,4,South,1419,,,,,,,
FIPS,int64,0,2868,2868.0,,,,31029.039052,15411.398662,1001.0,18102.5,30110.0,46123.5,56045.0
ElectionDate,object,0,18,2868.0,18,2/5/2008,1128,,,,,,,
ElectionType,object,0,2,2868.0,2,Primary,2558,,,,,,,
TotalVote,float64,1131,1490,1737.0,,,,12863.7962,53071.884245,13.0,732.0,2330.0,6418.0,1413869.0
Clinton,float64,1131,1338,1737.0,,,,5973.738054,25402.029497,4.0,329.0,1106.0,3200.0,771700.0
Obama,float64,1131,1314,1737.0,,,,6178.122625,27477.231157,4.0,254.0,878.0,2749.0,743686.0
MalesPer100Females,float64,0,391,2868.0,,,,99.081729,9.276792,76.2,94.9,97.6,100.5,200.9


In [222]:
elect_df["ElectionDate"] = pd.to_datetime(elect_df["ElectionDate"])

In [223]:
elect_df_train = elect_df[elect_df["ElectionDate"] < pd.to_datetime("2/19/2008")].copy()

In [224]:
elect_df_test = elect_df[elect_df["ElectionDate"] >= pd.to_datetime("2/19/2008")].copy()

In [225]:
elect_df_train["Obama_margin"] = elect_df_train["Obama"] - elect_df_train["Clinton"]
elect_df_train["Obama_margin_percent"] = elect_df_train["Obama_margin"] / elect_df_train["TotalVote"]
elect_df_train["Obama_wins"] = elect_df_train["Obama_margin"] > 0
elect_df_test["Obama_margin"] = elect_df_test["Obama"] - elect_df_test["Clinton"]
elect_df_test["Obama_margin_percent"] = elect_df_test["Obama_margin"] / elect_df_test["TotalVote"]
elect_df_test["Obama_wins"] = elect_df_test["Obama_margin"] > 0

In [226]:
summarize_dataframe(elect_df_test)

,Data Type,Missing Values,Unique Values,count,unique,top,freq,mean,min,25%,50%,75%,max,std
County,object,0,814,1131.0,814,Washington,11,,,,,,,
State,object,0,16,1131.0,16,TX,251,,,,,,,
Region,object,0,4,1131.0,4,South,608,,,,,,,
FIPS,int64,0,1131,1131.0,,,,38894.412025,15001.0,28134.0,41047.0,48268.0,56045.0,11922.301641
ElectionDate,datetime64[ns],0,9,1131.0,,,,2008-04-10 02:04:46.472148480,2008-02-19 00:00:00,2008-03-04 00:00:00,2008-04-22 00:00:00,2008-05-13 00:00:00,2008-06-03 00:00:00,
ElectionType,object,0,2,1131.0,2,Primary,1104,,,,,,,
TotalVote,float64,1131,0,0.0,,,,,,,,,,
Clinton,float64,1131,0,0.0,,,,,,,,,,
Obama,float64,1131,0,0.0,,,,,,,,,,
MalesPer100Females,float64,0,258,1131.0,,,,98.828382,76.2,94.9,97.7,100.2,194.6,8.417586


In [227]:
elect_df_validation = train_test_split(elect_df_test, train_size=.25, random_state=201)

In [228]:
summarize_dataframe(elect_df_test)

,Data Type,Missing Values,Unique Values,count,unique,top,freq,mean,min,25%,50%,75%,max,std
County,object,0,814,1131.0,814,Washington,11,,,,,,,
State,object,0,16,1131.0,16,TX,251,,,,,,,
Region,object,0,4,1131.0,4,South,608,,,,,,,
FIPS,int64,0,1131,1131.0,,,,38894.412025,15001.0,28134.0,41047.0,48268.0,56045.0,11922.301641
ElectionDate,datetime64[ns],0,9,1131.0,,,,2008-04-10 02:04:46.472148480,2008-02-19 00:00:00,2008-03-04 00:00:00,2008-04-22 00:00:00,2008-05-13 00:00:00,2008-06-03 00:00:00,
ElectionType,object,0,2,1131.0,2,Primary,1104,,,,,,,
TotalVote,float64,1131,0,0.0,,,,,,,,,,
Clinton,float64,1131,0,0.0,,,,,,,,,,
Obama,float64,1131,0,0.0,,,,,,,,,,
MalesPer100Females,float64,0,258,1131.0,,,,98.828382,76.2,94.9,97.7,100.2,194.6,8.417586


In [229]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

In [230]:
summarize_dataframe(elect_df_test)

,Data Type,Missing Values,Unique Values,count,unique,top,freq,mean,min,25%,50%,75%,max,std
County,object,0,814,1131.0,814,Washington,11,,,,,,,
State,object,0,16,1131.0,16,TX,251,,,,,,,
Region,object,0,4,1131.0,4,South,608,,,,,,,
FIPS,int64,0,1131,1131.0,,,,38894.412025,15001.0,28134.0,41047.0,48268.0,56045.0,11922.301641
ElectionDate,datetime64[ns],0,9,1131.0,,,,2008-04-10 02:04:46.472148480,2008-02-19 00:00:00,2008-03-04 00:00:00,2008-04-22 00:00:00,2008-05-13 00:00:00,2008-06-03 00:00:00,
ElectionType,object,0,2,1131.0,2,Primary,1104,,,,,,,
TotalVote,float64,1131,0,0.0,,,,,,,,,,
Clinton,float64,1131,0,0.0,,,,,,,,,,
Obama,float64,1131,0,0.0,,,,,,,,,,
MalesPer100Females,float64,0,258,1131.0,,,,98.828382,76.2,94.9,97.7,100.2,194.6,8.417586


In [240]:
lm_1 = smf.ols(formula='Obama_margin_percent ~ Region + Black + HighSchool + Poverty + PopDensity + SpeakingNonEnglish + LandArea', data=elect_df_test).fit()

ValueError: zero-size array to reduction operation maximum which has no identity

In [ ]:
variables = [variable for variable in elect_df_test.columns if variable not in ['FIPS', 'County', 'State', 'ElectionDate', 'TotalVote', 'Obama_margin', 'Obama_margin_percent', 'Obama_wins', 'Clinton', 'Obama']]

In [ ]:
variables

In [ ]:
formula = 'Obama_margin_percent ~ ' + "+ ".join(variables[:2]) +  " + standardize(" + ") + standardize(".join(variables[2:]) + ")"

In [ ]:
print(formula)

In [ ]:
y_linear, X_linear = dmatrices(formula, elect_df_test, return_type="dataframe")

In [ ]:
X_linear

In [ ]:
y_linear

In [ ]:
## Engineer features

In [ ]:
## Often when we are building models, we may want to transform the data in some way that doesn't rely on past or future data. This might be something like multiplying two columns together to create an interaction. Alternatively, it might be creating a new variable to indicate whether a state has some special meaning for either Clinton or Obama. As long as we can create this new feature using only a single row of data, we can go ahead and do that now.

In [ ]:
## However, if we instead are creating a new feature where we have to use data from other rows, this is a learning problem, and we should wait until we have a smaller training and validation sets set up, which we will do in the next section. An example of this might be if we wanted to create a new feature that was the average number of `MalesPer100Females` for the whole training data set. If we did this, we would be including information that we should not have yet in this average. Specifically, we would be using data that we are trying to predict on to compute the average, and the average is naturally going to include information about this data. This is bad practice when building a model because you obviously cannot do this when you are deploying a model in practice. There is no way to use data that doesn't exist yet to predict that data!

In [ ]:
## I will not include an specific new features here, but when you are building a model for the competition, you may want to. I'll leave a blank cell to invite you to do so.

In [ ]:
lm_1 = smf.ols(formula='Obama_margin_percent ~ Region + Black + HighSchool + Poverty + PopDensity + SpeakingNonEnglish + LandArea', data=elect_df_test).fit()